In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# ENABLE IF USING COLAB
USE_COLAB = False

if USE_COLAB:
    import os
    from google.colab import drive
    
    FOLDERNAME = 'UMass_Amherst/NN_AMI/NN_Project'
    drive.mount('/content/drive')
    %cd /content/drive/My\ Drive/$FOLDERNAME

    if os.path.exists('./spectroconv/'):
      %cd ./spectroconv
      !pip install hub
      !pip install hub[audio]
      !pip install wandb
    else:
      print("Error, must first pull project repository from github to Drive using ColabGitCommands.ipynb")

In [3]:
import torch
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: nvidia-smi: command not found


In [7]:
from models import wb_experiment

In [21]:
n_mels = 128
wb_config = {
    'project': 'spectroconv-debug',
    'entity': 'jakeval-colab',
    'group': 'cnn' # use this only when training/evaluating/etc a CNN
}

config = {
    'data': {
        'train': {
            'name': 'nsynth-med-train'
        },
        'val': {
            'name': 'nsynth-med-val'
        }
    },
    'model': {
        'id': wb_experiment.ModelType.CNN,
        'name': 'CNN-small'
    },
    'logging': {
        'batch_log_interval': 50,
        'eval_every_epoch': True,
    },
    'parameters': {
      'epochs': 2,
      'batch_size': 64,
      'optimizer': 'adam',
      'learning_rate': 0.0005,
      'dropout_fc': 0.5, # probability of an element to be zeroed (0 = no dropout)
      'dropout_conv': 0.1,
      'dropout_input': 0.2,
      'l2_reg': 0,
      'kernel_size': 3,
      'num_channels': [8, 16],
      'linear_layer_size': 32,
    },
}

In [ ]:
trainer = wb_experiment.TrainExperiment(wb_config=wb_config, login_key=False)
trainer.run_train(config, save_model=False)

hub://jakeval/nsynth-med-train-metadata loaded successfully.


/home/jakeval/umass/682/project/spectroconv/.env/lib/python3.8/site-packages/hub/core/fast_forwarding.py:43: UserWarning: Loading a dataset that was created or updated with a newer version of hub. This could lead to corruption or unexpected errors! Dataset version: 2.4.0, current hub version: 2.3.2. It's recommended that you update to a version of hub >= 2.4.0.
  warnings.warn(


This dataset can be visualized at https://app.activeloop.ai/jakeval/nsynth-med-train-metadata.
hub://jakeval/nsynth-med-train loaded successfully.
This dataset can be visualized at https://app.activeloop.ai/jakeval/nsynth-med-train.
hub://jakeval/nsynth-med-val-metadata loaded successfully.
This dataset can be visualized at https://app.activeloop.ai/jakeval/nsynth-med-val-metadata.
hub://jakeval/nsynth-med-val loaded successfully.
This dataset can be visualized at https://app.activeloop.ai/jakeval/nsynth-med-val.


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/953 [00:00<?, ?it/s]

In [ ]:
# untested

config = {
    'data': {
        'val': {
            'name': f'nsynth-full-{n_mels}-val'
        }
    },
    'model_name': 'CNN',
    'eval': {
        'examples': {
            'k': 8
        }
    }
}


In [ ]:
trainer = wb_experiment.EvalExperiment(wb_config)
metrics, examples_df = trainer.run_evaluate(config)

## Train on the full dataset

In [ ]:
# Wandb should take care of plotting for us?

'''
from matplotlib import pyplot as plt
fig, ax = plt.subplots()
ax.plot(losses)
fig, ax = plt.subplots()
ax.plot(train_accuracies, label='train acc')
ax.plot(val_accuracies, label='val acc')
ax.plot(val_f1s, label='val f1')
ax.legend()
plt.savefig("learning_metrics.png")
plt.show()
'''

In [ ]:
#print(train_model.stream_accuracy(clf, dl_val, device))

0.7415788789179347


In [ ]:
#print(train_model.stream_f1(clf, dl_val, device))

0.7109029348873568
